<a href="https://colab.research.google.com/github/ravimashru/100-days-of-deep-learning/blob/master/docs/days/023_Custom_Losses_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Losses in TensorFlow

TensorFlow has implementations of all popular loss functions you would need to use most of the time.

However, once in a while, you may need to create your own loss function - either to experiement, do research of your own, or implement a new loss function that you read about in a research paper.

There are two ways of creating custom losses in TensorFlow: as a function and as a class (that subclasses `tf.keras.losses.Loss`).

In this notebook, we'll see how to use both methods to create a custom Huber loss (Disclosure: TensorFlow already has [an implementation for the Huber loss](https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/losses.py#L1095-L1159): `tf.keras.losses.Huber`. However, we'll pretend it doesn't exist so that we can learn how to create our own custom losses).

## The Huber Loss

[The Huber loss](https://en.wikipedia.org/wiki/Huber_loss) between actual and predicted labels is like a hybrid of a squared loss and a linear loss.

If the absolute difference between the true and predicted label is below a predefined **threshold**, then it behaves like squared loss and if it is above the **threshold** it behaves like linear loss.

Formally,

$$
\text{HuberLoss}(y_{true}, y_{pred}) = 
\begin{cases}
  \frac{1}{2} \cdot {(y_{true} - y_{pred})}^2,
  & \text{if } y_{true} - y_{pred} < \delta\\
  \delta |y_{true} - y_{pred}| - \frac{1}{2} \delta^2, & \text{otherwise}
\end{cases}
$$

where $\delta$ is the threshold.

## Create a baseline

We will use the inbuilt mean square error loss function to create a baseline model on the California housing dataset.


In [1]:
import tensorflow as tf

print('TensorFlow: ', tf.__version__)
print('Keras: ', tf.keras.__version__)

TensorFlow:  2.3.0
Keras:  2.4.0


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# This dataset contains only numerical features and has no missing values
housing = fetch_california_housing()

# Train/validation/test split
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

# Create and train model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
  tf.keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer="adam")

model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
print(mse_test)

Epoch 1/20
363/363 [==============================] - 1s 3ms/step - loss: 10778.4756 - val_loss: 13.2174
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 54.5468 - val_loss: 6.1404
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 39.4096 - val_loss: 3.8407
Epoch 4/20
363/363 [==============================] - 1s 1ms/step - loss: 29.7426 - val_loss: 2.3915
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 18.7482 - val_loss: 2.5969
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 11.7517 - val_loss: 1.3069
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 7.4931 - val_loss: 1.2469
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 4.5499 - val_loss: 0.9910
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 2.4598 - val_loss: 0.9339
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 1.3431 - val_lo

## Create a custom loss function

Using a custom loss function in TensorFlow is as easy as creating a function that has two parameters: $y_{true}$ and $y_{pred}$.

In [3]:
# Define custom Huber loss function
# Takes threshold as a parameter
# Returns function that calculates Huber loss with given threshold
def make_huber_loss(threshold=1.0):

  def huber_loss_fn(y_true, y_pred):

    # Calculate difference between actual and predicted
    delta = y_true - y_pred

    # Mark which differences are below threshold
    below_threshold = tf.abs(delta) < threshold

    # Calculated squared loss (used for delta < threshold)
    squared_loss = 0.5 * tf.square(delta)

    # Calculate linear loss (used for delta > threshold)
    linear_loss = threshold * tf.abs(delta) - 0.5 * threshold**2

    # Return loss based on delta and threshold
    return tf.where(below_threshold, squared_loss, linear_loss)

  return huber_loss_fn

# Use the custom loss function to train the model
custom_huber_loss = make_huber_loss()

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
  tf.keras.layers.Dense(1)
])

model.compile(loss=custom_huber_loss, optimizer="adam")

model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
print(mse_test)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 7.1424 - val_loss: 0.5724
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.6046 - val_loss: 0.5325
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5913 - val_loss: 0.3536
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5497 - val_loss: 0.3919
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5500 - val_loss: 0.6900
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5480 - val_loss: 0.3744
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3940 - val_loss: 0.5724
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5156 - val_loss: 0.2885
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4469 - val_loss: 0.2729
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5020 - val_loss: 0.3772

## Create a custom loss class

To create a custom loss class, you have to subclass `tf.keras.losses.Loss`. You need to override the `call` method with your custom loss function code.

In [6]:
# Create custom loss class
class CustomHuberLoss(tf.keras.losses.Loss):

  def __init__(self, threshold=1.0, **kwargs):
    self.threshold = threshold
    super().__init__(**kwargs)

  # This method calculates and returns the loss
  def call(self, y_true, y_pred):
    delta = y_true - y_pred
    below_threshold = tf.abs(delta) < self.threshold
    squared_loss = 0.5 * tf.square(delta)
    linear_loss = self.threshold * tf.abs(delta) - 0.5 * self.threshold**2
    return tf.where(below_threshold, squared_loss, linear_loss)

# Use the custom loss function to train the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
  tf.keras.layers.Dense(1)
])

model.compile(loss=CustomHuberLoss(), optimizer="adam")

model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
print(mse_test)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 7.5532 - val_loss: 0.4532
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5518 - val_loss: 0.8375
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4469 - val_loss: 0.7997
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5783 - val_loss: 0.3615
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3717 - val_loss: 0.3331
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4114 - val_loss: 0.3781
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4651 - val_loss: 0.3141
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4824 - val_loss: 0.2808
Epoch 9/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4401 - val_loss: 0.4061
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4628 - val_loss: 1.0965